<a href="https://colab.research.google.com/github/harry-hrz/torch-image-classification/blob/main/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.utils.data as torchdata
from tqdm import tqdm

## LeNet

In [35]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, 5), 
            nn.ReLU(), 
            nn.AvgPool2d(2, 2), 
            nn.Conv2d(6, 16, 5), 
            nn.ReLU(), 
            nn.AvgPool2d(2, 2), 
            nn.Conv2d(16, 120, 5), 
            nn.ReLU()
        )
        self.fc1 = nn.Linear(120*1*1, 84)
        self.fc2 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## train

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

transform = transforms.Compose([
    transforms.Resize((32, 32)), 
    transforms.ToTensor(), 
    transforms.Normalize((0.1307), (0.3081))
])
train_set = torchvision.datasets.MNIST(train=True, transform=transform, download=True, root='./data')
train_loader = torchdata.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=0)

val_set = torchvision.datasets.MNIST(train=False, transform=transform, download=True, root='./data')
val_loader = torchdata.DataLoader(val_set, batch_size=10000, num_workers=0)
val_data_iter = iter(val_loader)
val_image, val_label = next(val_data_iter)

model = LeNet()
model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=10e-3)

for epoch in range(10):
    run_loss = 0.0
    for step, data in tqdm(enumerate(train_loader)):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = loss_func(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        run_loss += loss.item()
    with torch.no_grad():
        outputs = model(val_image.to(device))
        predict = torch.max(outputs, dim=1)[1]
        acc = (predict == val_label.to(device)).sum() / val_label.shape[0]
    print('epoch[%d], train_loss: %.3f, val_acc: %.3f' % (epoch+1, run_loss/step, acc))
print('Finished training')